In [1]:
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV

from xgboost import XGBClassifier

In [2]:
data = pd.read_csv('train_jqd04QH.csv')
test = pd.read_csv('test_GYi4Gz5.csv')

In [3]:
# #Replacing Null values of Company_type to Other
data['company_type'].fillna('NpO', inplace = True)
test['company_type'].fillna('NpO', inplace = True)

#replacing with medium company size
data['company_size'].fillna('50-99', inplace  = True)
test['company_size'].fillna('50-99', inplace  = True)

# # #Managing all values accordingly
data.experience[data.experience == '>20'] = 20 
data.experience[data.experience == '<1'] = 1 

test.experience[test.experience == '>20'] = 20 
test.experience[test.experience == '<1'] = 1 

# #Replacing NULL values in experience with median
data['experience'].fillna(9, inplace = True)
test['experience'].fillna(9, inplace = True)

# #Replacing all values in major decipline with 'Other
data['major_discipline'].fillna('Other', inplace = True)
test['major_discipline'].fillna('Other', inplace = True)

# #Handling all the values 
data.last_new_job[data.last_new_job == '>4'] = 5 
test.last_new_job[test.last_new_job == '>4'] = 5 

# #Replscing all the NULL values with Mode
data['last_new_job'].fillna(2, inplace = True)
test['last_new_job'].fillna(2, inplace = True)

# #Replacing all the NULL values in education_evel with mode
data['education_level'].fillna('Masters', inplace = True)
test['education_level'].fillna('Masters', inplace = True) 

# #Replacing all the NULL values in enrolled university with mode
data['enrolled_university'].fillna('Part time course', inplace = True)
test['enrolled_university'].fillna('Part time course', inplace = True)

# #Filling NULL values with mode
data.gender.fillna('Other', inplace =  True) #data['gender'].mode()[0]
test.gender.fillna('Other', inplace =  True) #test['gender'].mode()[0]

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs

In [4]:
X = data.drop(['target'] ,axis = 1)
y = data['target'].copy()
X_test = test

In [5]:
from sklearn import preprocessing

for i in X.columns:
    if X[i].dtype == 'object':
        if len(list(X[i].unique())) >=2:
            #cols.append(i)
            le = preprocessing.LabelEncoder()
            le.fit(X[i].astype(str))
            X[i] = le.transform(X[i].astype(str))
            X_test[i] = le.transform(X_test[i].astype(str))
            #X_val[i] =   le.transform(X_val[i].astype(str))
            #X_test[i] =  le.transform(X_test[i].astype(str))

In [6]:


X = pd.concat([X.get(['enrollee_id','city','city_development_index', 'gender', 'relevent_experience', 'experience',
                                                                            'last_new_job', 'training_hours' ]),
                           pd.get_dummies(X['enrolled_university'], prefix='enrolled_university'),
                           pd.get_dummies(X['education_level'], prefix='education_level'),
                           pd.get_dummies(X['major_discipline'], prefix='major_discipline'),
                           pd.get_dummies(X['company_size'], prefix='company_size'),
                           pd.get_dummies(X['company_type'], prefix='company_type')],axis=1)

X_test = pd.concat([X_test.get(['enrollee_id','city','city_development_index', 'gender', 'relevent_experience', 'experience',
                                                                            'last_new_job', 'training_hours' ]),
                           pd.get_dummies(X_test['enrolled_university'], prefix='enrolled_university'),
                           pd.get_dummies(X_test['education_level'], prefix='education_level'),
                           pd.get_dummies(X_test['major_discipline'], prefix='major_discipline'),
                           pd.get_dummies(X_test['company_size'], prefix='company_size'),
                           pd.get_dummies(X_test['company_type'], prefix='company_type')],axis=1)

# GBM

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection  import GridSearchCV

In [8]:
est = GradientBoostingClassifier(n_estimators = 1800, learning_rate=0.01, max_depth = 3,
                                 min_samples_leaf = 10, max_features = 0.25)

In [9]:
est.fit(X,y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=3,
              max_features=0.25, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=10, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=1800,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [10]:
preds = est.predict_proba(X_test)

In [11]:
sub = pd.read_csv('sample_submission_sxfcbdx.csv')

In [12]:
sub['target'] = preds[:,1]

In [13]:
subd = sub.to_csv('4.csv', index = False)